In [156]:
%pip install nltk
%pip install pandas
%pip install scikit-learn

import csv
from nltk.translate.bleu_score import sentence_bleu
import string 
import re 

import string

import pandas as pd
import nltk
from nltk.corpus import wordnet

nltk.download('punkt')


from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
import re
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /home/imranm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/imranm3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/imranm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/imranm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/imranm3/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [157]:


lemmatizer = WordNetLemmatizer()

sw = stopwords.words('english')
porter_stemmer = PorterStemmer()


def text_cleaning(text):
    text = str (text)

    printable = set(string.printable)
    text = ''.join(filter(lambda x: x in printable, text))
    text = text.replace('\x00', ' ')  # remove nulls
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = text.lower()  # Lowercasing

    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    wn_tags = {'N': wordnet.NOUN, 'V': wordnet.VERB, 'R': wordnet.ADV, 'J': wordnet.ADJ}
    # lemmatize each word with its POS tag
    lemmatized_words = []
    for word, pos in pos_tags:
        if pos[0] in wn_tags:
            wn_tag = wn_tags[pos[0]]
            lemmatized_words.append(lemmatizer.lemmatize(word, wn_tag))
        else:
            lemmatized_words.append(word)
    text = ' '.join([str(elem) for elem in lemmatized_words])

    words= text.split()
    stemmed_words=[porter_stemmer.stem(word=word) for word in words] # Stemming
    text = ' '.join([str(elem) for elem in stemmed_words])

    text = text.strip()
    return text


In [158]:
column_comments = []
column_gold = []
column_chatgpt = []
column_gpt4 = []
column_flan = []


with open('causes.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    idx = 0
    for row in csv_reader:
        column_comments.append(text_cleaning(row[1]))
        column_gold.append(text_cleaning(row[2]))
        column_chatgpt.append(text_cleaning(row[4]))
        column_gpt4.append(text_cleaning(row[5]))
        column_flan.append(text_cleaning(row[6]))

print(len(column_gold))

450


In [159]:
word_counter = 0
for sentence in column_comments:
    word_counter = word_counter + len(sentence.split())
print(word_counter/len(column_flan))

28.08888888888889


In [160]:
word_counter = 0
for sentence in column_gold:
    word_counter = word_counter + len(sentence.split())
print(word_counter/len(column_flan))

7.428888888888889


In [161]:
word_counter = 0
for sentence in column_gpt4:
    word_counter = word_counter + len(sentence.split())
print(word_counter/len(column_flan))

8.846666666666666


In [162]:
word_counter = 0
for sentence in column_chatgpt:
    word_counter = word_counter + len(sentence.split())
print(word_counter/len(column_flan))

8.642222222222221


In [163]:
word_counter = 0
for sentence in column_flan:
    word_counter = word_counter + len(sentence.split())
print(word_counter/len(column_flan))

13.115555555555556


In [164]:
# ChatGPT: Calculate the BLEU score for each row in the columns
chatgpt_scores = []
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_chatgpt[i], weights=(1.0, 0, 0, 0))
    # print(score)
    scores.append(score)


print(len(scores))

avg_score = sum(scores) / len(scores)
chatgpt_scores.append(avg_score)
print(f"Average BLEU-1 score: {avg_score}")

450
Average BLEU-1 score: 0.5216223155541132


/home/imranm3/Desktop/research/SE-Emotion-Cause/venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/imranm3/Desktop/research/SE-Emotion-Cause/venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/imranm3/Desktop/research/SE-Emotion-Cause/venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how m

In [165]:
# ChatGPT: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_chatgpt[i], weights=(0.5, 0.5))
    # print(score)
    scores.append(score)


chatgpt_blue_2_scores = scores    


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
chatgpt_scores.append(avg_score)
print(f"Average BLEU-2 score: {avg_score}")

450
Average BLEU-2 score: 0.48862693512857797


In [166]:
# ChatGPT: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_chatgpt[i], weights=(0.33, 0.33, 0.33))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
chatgpt_scores.append(avg_score)
print(f"Average BLEU-3 score: {avg_score}")

450
Average BLEU-3 score: 0.4666842196987067


In [167]:
# ChatGPT: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_chatgpt[i], weights=(0.25, 0.25, 0.25, 0.25))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
chatgpt_scores.append(avg_score)
print(f"Average BLEU-4 score: {avg_score}")

450
Average BLEU-4 score: 0.4504333236694316


In [168]:
# GPT-4: Calculate the BLEU score for each row in the columns
scores = []
gpt4_scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_gpt4[i], weights=(1.0, 0, 0, 0))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
gpt4_scores.append(avg_score)
print(f"Average BLEU-1 score: {avg_score}")

450
Average BLEU-1 score: 0.6373626036317592


In [169]:
# GPT-4: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_gpt4[i], weights=(0.5, 0.5))
    # print(score)
    scores.append(score)

gpt4_blue_2_scores = scores

print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
gpt4_scores.append(avg_score)
print(f"Average BLEU-2 score: {avg_score}")

450
Average BLEU-2 score: 0.5975115667777628


In [170]:
# GPT-4: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_gpt4[i], weights=(0.33, 0.33, 0.33))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
gpt4_scores.append(avg_score)
print(f"Average BLEU-3 score: {avg_score}")

450
Average BLEU-3 score: 0.5711087979153057


In [171]:
# GPT-4: Calculate the BLEU score for each row in the columns
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_gpt4[i], weights=(0.25, 0.25, 0.25, 0.25))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
gpt4_scores.append(avg_score)
print(f"Average BLEU-4 score: {avg_score}")

450
Average BLEU-4 score: 0.5543213195547464


In [172]:
# flan-alpaca: Calculate the BLEU score for each row in the columns
flan_alpaca_scores = []
scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_flan[i], weights=(1.0, 0, 0, 0))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
flan_alpaca_scores.append(avg_score)
print(f"Average BLEU-1 score: {avg_score}")

450
Average BLEU-1 score: 0.5710519279089656


In [173]:
# flan-alpaca: Calculate the BLEU score for each row in the columns

scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_flan[i], weights=(0.5, 0.5))
    # print(score)
    scores.append(score)

flan_blue_2_scores = scores

print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
flan_alpaca_scores.append(avg_score)
print(f"Average BLEU-2 score: {avg_score}")

450
Average BLEU-2 score: 0.5428681782625324


In [174]:
# flan-alpaca: Calculate the BLEU score for each row in the columns

scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_flan[i], weights=(0.33, 0.33, 0.33))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
flan_alpaca_scores.append(avg_score)
print(f"Average BLEU-3 score: {avg_score}")

450
Average BLEU-3 score: 0.524756866685228


In [175]:
# flan-alpaca: Calculate the BLEU score for each row in the columns

scores = []
for i in range(len(column_gold)):
    score = sentence_bleu([column_gold[i]], column_flan[i], weights=(0.25, 0.25, 0.25, 0.25))
    # print(score)
    scores.append(score)


print(len(scores))

# Calculate the average BLEU score for the columns
avg_score = sum(scores) / len(scores)
flan_alpaca_scores.append(avg_score)
print(f"Average BLEU-4 score: {avg_score}")

450
Average BLEU-4 score: 0.5077396989640749


In [176]:
print(chatgpt_scores)
print(gpt4_scores)
print(flan_alpaca_scores)

[0.5216223155541132, 0.48862693512857797, 0.4666842196987067, 0.4504333236694316]
[0.6373626036317592, 0.5975115667777628, 0.5711087979153057, 0.5543213195547464]
[0.5710519279089656, 0.5428681782625324, 0.524756866685228, 0.5077396989640749]


In [181]:
count = 0
word_counter_comments = 0
word_counter_gold = 0
word_counter_chatgpt = 0
word_counter_gpt4 = 0
word_counter_flan = 0

for i in range(len(flan_blue_2_scores)):
    if flan_blue_2_scores[i] < 0.3 and chatgpt_blue_2_scores[i] < 0.3 and gpt4_blue_2_scores[i] < 0.3:
        count = count + 1
        word_counter_comments = word_counter_comments + len(column_comments[i].split())
        word_counter_gold = word_counter_gold + len(column_gold[i].split())
        word_counter_gpt4 = word_counter_gpt4 + len(column_gpt4[i].split())
        word_counter_chatgpt = word_counter_chatgpt + len(column_chatgpt[i].split())
        word_counter_flan = word_counter_flan + len(column_flan[i].split())
print(count)

print(word_counter_comments/count)
print(word_counter_gold/count)
print(word_counter_gpt4/count)
print(word_counter_chatgpt/count)
print(word_counter_flan/count)

41
44.170731707317074
5.048780487804878
10.097560975609756
13.146341463414634
22.829268292682926


In [180]:
count = 0
word_counter_comments = 0
word_counter_gold = 0
word_counter_chatgpt = 0
word_counter_gpt4 = 0
word_counter_flan = 0

for i in range(len(flan_blue_2_scores)):
    if flan_blue_2_scores[i] > 0.5 and chatgpt_blue_2_scores[i] > 0.5 and gpt4_blue_2_scores[i] > 0.5:
        count = count + 1
        word_counter_comments = word_counter_comments + len(column_comments[i].split())
        word_counter_gold = word_counter_gold + len(column_gold[i].split())
        word_counter_gpt4 = word_counter_gpt4 + len(column_gpt4[i].split())
        word_counter_chatgpt = word_counter_chatgpt + len(column_chatgpt[i].split())
        word_counter_flan = word_counter_flan + len(column_flan[i].split())
print(count)

print(word_counter_comments/count)
print(word_counter_gold/count)
print(word_counter_gpt4/count)
print(word_counter_chatgpt/count)
print(word_counter_flan/count)

107
15.261682242990654
7.02803738317757
7.88785046728972
7.794392523364486
7.953271028037383
